In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

In [4]:
x = yf.Ticker("MSFT")
x.cashflow.loc["Free Cash Flow",:]
x.info["sector"]

'Technology'

In [179]:
x.info.get('industry')
#x.balance_sheet.index

'Software - Infrastructure'

In [5]:
def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x


def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x


def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

### Ebit



In [6]:
n = 4

ebit = x.income_stmt.loc["EBIT",:].sort_index()
g = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g)**i) for i in range(n+1)], index=range(0,n+1))[1:]
ebit_forecast

1    1.159019e+11
2    1.517487e+11
3    1.986824e+11
4    2.601321e+11
dtype: float64

### Tax

In [7]:
tax_rate = x.incomestmt.loc["Tax Rate For Calcs"]
tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]

### Change in KTN

In [8]:
# 1.- Directly
ktn_delta = x.cashflow.loc["Change In Working Capital", :]
ktn_delta 

2023-06-30   -2388000000.0
2022-06-30     446000000.0
2021-06-30    -936000000.0
2020-06-30    2148000000.0
Name: Change In Working Capital, dtype: object

In [9]:
# 2.- With ratios
usa_ktn_percentage = get_working_capital("").loc[sector,"Non-cash WC/ Sales"]
sector = "Software (System & Application)"

apr = get_growth_historical("").loc[sector,"Expected Growth in Revenues - Next 2 years"]
g = (1+apr)**(1/2)-1


sales = x.incomestmt.loc["Total Revenue",:].sort_index()
sales_forecast = pd.Series([sales.iloc[-1]*((1+g)**i) for i in range(n+1)], index=range(0,n+1))
sales_forecast

ktn_forecast = sales_forecast * usa_ktn_percentage
ktn_delta_forecast = ktn_forecast.diff()[1:]
ktn_delta_forecast 

1    2.947163e+09
2    3.261794e+09
3    3.610015e+09
4    3.995411e+09
dtype: float64

### Capex - Dep

In [10]:
# 1- 


In [11]:
capex_vs_sales = get_capex("").loc[sector, "Net Cap Ex/Sales"]
capex_net = (sales_forecast * capex_vs_sales)[1:]
capex_net

1    4.996929e+10
2    5.530389e+10
3    6.120799e+10
4    6.774240e+10
dtype: float64

# FCF 

In [12]:
ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast

1    4.237772e+10
2    6.620163e+10
3    9.853806e+10
4    1.421420e+11
dtype: float64

## Method 1:
- ebit: EBIT increases at an expected rate, this rate is extracted from Damodaran for sector
- Tax: Tax rate is extracted from damodaran, it represent the average within the industry an oly accounts those companies that had profits
- CAPEX net: CAPEX NET 

In [13]:
n = 4
sector = "Software (System & Application)"

ebit = x.income_stmt.loc["EBIT",:].sort_index()
g_ebit = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g)**i) for i in range(n+1)], index=range(0,n+1))[1:]

tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]

usa_ktn_percentage = get_working_capital("").loc[sector,"Non-cash WC/ Sales"]
sales_growth_2y = get_growth_historical("").loc[sector,"Expected Growth in Revenues - Next 2 years"]
g_sales = (1+sales_growth_2y)**(1/2)-1

sales = x.incomestmt.loc["Total Revenue",:].sort_index()
sales_forecast = pd.Series([sales.iloc[-1]*((1+g)**i) for i in range(n+1)], index=range(0,n+1))
sales_forecast

ktn_forecast = sales_forecast * usa_ktn_percentage
ktn_delta_forecast = ktn_forecast.diff()[1:]


ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast

1    2.763704e+10
2    3.058750e+10
3    3.385294e+10
4    3.746699e+10
dtype: float64

# WACC